In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.font_manager as fm
import re
import scipy.stats


df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1)
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [3]:
print(df.info())
print("------------")
print(df.shape)
#df의 상태가 이상합니다. 우선 결측치는 Year  ,  Genre  ,  Publisher 컬럼에 존재합니다.
#출고량 데이터는 numeric 이어야합니다. onj인걸 보니 혹시 문자가 포함된건가..?의심해봅니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.1+ MB
None
------------
(16598, 9)


In [4]:
df.isna().sum()
#결측치가 각각 271,50,58개로 전체 표본 수 대비 드랍해줘도 괜찮을 수준입니다. 
#하지만 생각없이 다 드랍해버리다간 전체의 몇%나 드랍하게될지 알 수 없으니 일단 드랍하지 않습니다.
#분석 범주에 따라 연도가 필요치 않은 분석이 있을 수도 있기도 하고요.

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [5]:
#일단 Year의 결측치는 '0'으로 대체해놓습니다.
#참고 링크: https://jimmy-ai.tistory.com/169
df.loc[df['Year'] != df['Year'], 'Year'] = 0

#잘 대체됐을지 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16598 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.1+ MB


In [6]:
#나머지 두개 컬럼은 호옥시라도 특정 연도에만 결측치가 많고, 그 연도에 출고량이 극단적으로 많았다면 3번 과제의 분석 결과에 영향을 주게될수도 있으니까, 
#결측치를 0으로 대체하고 살려두기로 했습니다.

df.loc[df['Genre'] != df['Genre'], 'Genre'] = 0
df.loc[df['Publisher'] != df['Publisher'], 'Publisher'] = 0
#잘 되었는지 확인
print(df.Genre.value_counts())
print("----------------")
print(df.Publisher.isna().sum())

Action          3305
Sports          2341
Misc            1734
Role-Playing    1483
Shooter         1308
Adventure       1280
Racing          1243
Platform         884
Simulation       865
Fighting         847
Strategy         680
Puzzle           578
0                 50
Name: Genre, dtype: int64
----------------
0


-수치인데 자료형이 obj로 되어있는 친구들이 거슬려 바꿔주려 했는데, 문자열이 포함되어 있어 진행되지 않았습니다.
-확인 결과 M,k라는 문자열들이 간헐적으로 있었습니다. 이를 numeric하게 풀어줍니다.(단위 통일)

이 경우 우리가 할 수 있는 것

>방법1. 단위가 없는 숫자들은 다 같은 범주(m or k중 하나)로 작성되었다고 간주
나머지 숫자들을 변환하여 같이 쓴다.
>> 게임 매출이 k 소수점 두 자리 단위는 상식적이지 않고, 검색해봐도 매출이
그보다는 높았을 것으로 나온다. 따라서 단위가 없는 것들은 M이 기준 단위였을 가능성이 높다.
>>또한, 단위가 없는 데이터들은 기본적으로 같은 단위일것이다. 제멋대로의 단위를 가졌다면 과제로 냈을 리가 없다. (유의미한 분석 자체가 어렵기 때문)


>방법2. m또는 k가 붙어있는것을 버린다. 그리고 단위가 없는 숫자들은 
다 같은 범주(m or k중 하나)로 작성되었다고 간주하고 계산한다.  
(근데 이 경우 이득이 무엇..?)
>>방법 1대로 하려면 분류를 3가지로 해야하는데, 코드가 줄어서 속도도 빨라지고 편하다?
>이 경우 M과 K를 가진 수치의 비율을 보고 판단하자.
>>엑셀로 찾아본 결과 (쉽고 빠른 엑셀의 유혹..)거의 800개의 레코드가 나오는데, 모수가 약 16600개니까. 드랍하기엔 비중이 너무 크다. 그냥 변환하는 방법 1을 채택하자.


In [7]:
#원래의 목적대로 출고량 데이터들을 numeric 자료형으로 형변환 시켜서 리턴합니다.
#위에서 결정한 방식대로 M과 K를 numeric 형태로 풀어주되, 숫자가 쓸데없이 기니까 기본 단위를 K로 잡습니다.
#m과 k를 풀어주는 코드의 참고 링크: https://aytekin.tistory.com/50

def unit(x):
  if str(x).find("K") != -1:
    return float(str(x).split("K")[0].replace('.',''))
  elif str(x).find("M") != -1:
    return float(str(x).split("M")[0].replace('.',''))*1000
  else:
    return float(str(x).replace('.',''))*1000
#단위가 너무 크기때문에 sales 관련 수치들의 최소단위 자체를 k로 바꿨어요.
#칼럼명에 이 정보를 반영하면 중복이 3번 발생하니, 시각화에만 이 사실을 써줄게요.
for col in ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']:
  df[col] = df[col].apply(unit)
print(df.dtypes)
df.head()


Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,4000.0,0.0,0.0,0.0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,17000.0,0.0,0.0,1000.0
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.0,0.0,2000.0,0.0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,4000.0,0.0,0.0,0.0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,12000.0,9000.0,0.0,4000.0


In [8]:
#이제 꼴도보기싫은 .0 친구들을 모두 떼어 깔끔히 정리해줍니다.
colnames = df.columns
df = df.astype(int,errors='ignore')

#잘 됐나 확인
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008,Action,Destineer,4000,0,0,0
1,The Munchables,Wii,2009,Action,Namco Bandai Games,17000,0,0,1000
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0,0,2000,0
3,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,4000,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,12000,9000,0,4000


In [9]:
#year의 거슬리는 부분(앞의 두 개 숫자가 지워진 것)도 처리해줍니다.
#먼저 year의 분포를 한번 봅시다.
df.Year.value_counts(bins = 30)
#와우 말도안되는 연도들이 있네요. 대충 1952년 미만의 데이터들은 최대 134부터 시작하는 데이터들이겠어요.

(1952.667, 2020.0]      16227
(-2.021, 67.333]          357
(67.333, 134.667]          14
(134.667, 202.0]            0
(1144.667, 1212.0]          0
(1885.333, 1952.667]        0
(1818.0, 1885.333]          0
(1750.667, 1818.0]          0
(1683.333, 1750.667]        0
(1616.0, 1683.333]          0
(1548.667, 1616.0]          0
(1481.333, 1548.667]        0
(1414.0, 1481.333]          0
(1346.667, 1414.0]          0
(1279.333, 1346.667]        0
(1212.0, 1279.333]          0
(1077.333, 1144.667]        0
(202.0, 269.333]            0
(942.667, 1010.0]           0
(875.333, 942.667]          0
(808.0, 875.333]            0
(740.667, 808.0]            0
(673.333, 740.667]          0
(606.0, 673.333]            0
(538.667, 606.0]            0
(471.333, 538.667]          0
(404.0, 471.333]            0
(336.667, 404.0]            0
(269.333, 336.667]          0
(1010.0, 1077.333]          0
Name: Year, dtype: int64

In [10]:
#세 자리수나 음수 데이터가 있는지 확인해봅시다.
num_of_not_three_digit = df['Year'][df['Year']<100].count()
neg = df['Year'][df['Year']<0].count()

print(num_of_not_three_digit)
print(neg)

#없습니다. 야호!

371
0


In [11]:
#위의 bins를 바탕으로 생각해봤을 때, 게임은 최소 1952년 이후에 출시됐습니다.
#이상치 데이터가 모두 두 자리임을 감안했을때 연도의 끝 두자리만을 기입했을 가능성이 높습니다.
#위의 두가지 사항을 감안하여, 50보다 큰 데이터는 1900년대의 데이터, 50보다 작은 데이터는 2000년대의 데이터로 간주하여 처리합니다. 
#연도 분류 참고 링크: https://tommyhouse.oopy.io/cs_eda
def correct_year(value):
  if (value < 1952) & (value > 50):
    value = value + 1900
    return value
  elif (value < 50) & (value != 0):
    value = value + 2000
    return value
  else:
    return value

df['Year'] = df['Year'].apply(correct_year).astype(pd.Int64Dtype()).astype(str)
#연도별 인기 게임 분류 및 시각화 과제를 위한 groupby를 위해 미리 str로 바꿔주었습니다.

#잘 변환되었나 확인해봅니다. 값이 0인것과 정상적인것 (네자리)를 합한 수 쓰기! 
num_of_zero = df['Year'][df['Year'] == '0'].count()
Years_greater_than_1000 = df.Year.str.len() == 4
num_of_four_digit = Years_greater_than_1000[Years_greater_than_1000].count()

sum = num_of_zero + num_of_four_digit
sum

#전체 레코드 수와 같아 잘 변환된 듯 합니다.

16598

In [12]:
#또 전처리 해야 할 것이 있는지 봅니다.
print(df.info())
df.sample(3)

'''
이번 미션에서 반드시 해야 할 일:
0. 데이터 정제 
지역별 선호 게임을 알 수 있고, 연도별 인기 장르를 알 수 있고, 출고량이 많은 게임을 알 수 있게 해야함.
1.지역별 선호 게임 장르 분류 및 시각화 > 지역별 상위 장르 시각화
2.연도별 인기 게임 분류 및 시각화 > 연도별로 지역 상관없이 상위 장르 시각화
3.출고량이 높은 게임들의 장르 및 연도 분류 및 시각화 > 출고량 상위 게임들을 뽑은 다음, 장르와 출시 연도 시각화
4.(보너스) 지역별로 게임이 많이 팔렸던 연도 시각화 > 지역별 출고량 상위 연도 선별 후 시각화
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         16598 non-null  object
 1   Platform     16598 non-null  object
 2   Year         16598 non-null  object
 3   Genre        16598 non-null  object
 4   Publisher    16598 non-null  object
 5   NA_Sales     16598 non-null  int64 
 6   EU_Sales     16598 non-null  int64 
 7   JP_Sales     16598 non-null  int64 
 8   Other_Sales  16598 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.1+ MB
None


'\n이번 미션에서 반드시 해야 할 일:\n0. 데이터 정제 \n지역별 선호 게임을 알 수 있고, 연도별 인기 장르를 알 수 있고, 출고량이 많은 게임을 알 수 있게 해야함.\n1.지역별 선호 게임 장르 분류 및 시각화 > 지역별 상위 장르 시각화\n2.연도별 인기 게임 분류 및 시각화 > 연도별로 지역 상관없이 상위 장르 시각화\n3.출고량이 높은 게임들의 장르 및 연도 분류 및 시각화 > 출고량 상위 게임들을 뽑은 다음, 장르와 출시 연도 시각화\n4.(보너스) 지역별로 게임이 많이 팔렸던 연도 시각화 > 지역별 출고량 상위 연도 선별 후 시각화\n'

In [13]:
#연도별 인기 장르를 계산할 때는, 그냥 전체 출고량 피쳐를 하나 더 만들어주는게 메모리를 훨씬 더 소모하는 길이라 좋아보입니다.
#전체 출고량 피쳐는 네 피쳐들을 합해서 알 수 있는 값이기에 굳이 만들어야 하나? 싶지만.. 
#자주 필요한 데이터고, 그때마다 네 컬럼의 엘리먼트들을 하나하나 합치는건 메모리 낭비니까요.
#feature engineering 고고싱
df['total_sum'] = df['NA_Sales']+df['EU_Sales']+df['JP_Sales']+df['Other_Sales']

In [14]:
#이 위로는 1차 전처리 존입니다.
#전처리존과 구분해주는 셀
#전처리존과 구분해주는 셀
#전처리존과 구분해주는 셀
#전처리존과 구분해주는 셀
#전처리존과 구분해주는 셀
#전처리존과 구분해주는 셀
#전처리존과 구분해주는 셀

-

이하는 PPT를 통해 발표했습니다.
PPT 첨부 링크: https://drive.google.com/drive/u/0/folders/1wq2E-aPrSJXcdnEgfqlxskIAp5fGdII4

-

- 데이터 분석 배경 상황: 일본에 본사를 둔 게임 업계 1인자 코드소테츠의 게임기획부에서
하반기 신작 미연시(미소녀 연애 시뮬레이션) 기획 및 구체화를 위한 하반기 신작 게임 제안서를 준비하고 있다.

- 데이터 분석 배경 세부 상황: 코드소테츠는 전세계 솔로 인구들의 행복을 위해 일본 미연시 사업에 진출했고,
작년 출시된 rpg 미연시 'AI 러브 부트! 15배 좋아해'가 초 대박을 쳐서
글로벌 시장 추가 진출을 결정, 게임 기획을 위한 데이터 분석에 착수했다.

- 분석 결과에 대한 1순위 고려 대상: 공격적인 성장 전략을 펼치고 있어, 가능한 돈 되는 시장에서 실패할 확률이 낮은 장르로 신작 게임을 출시하는 것이 1차 목표다.

- 분석 결과 진출하게 될 시장: 고려중인 시장은 북미,일본,유럽 및 기타 국가들이다.

In [ ]:
#과제 1번.지역별 선호 게임 장르 분류 및 시각화를 실천해봅시다.
#이는 국가별 시장의 니즈 분석을 위한 단계입니다.
#멜트해서 직관적이고..분류 속 요소들을 세부적으로 인지하기 쉬운 displot형식 그래프를 만들고싶었지만
#displot을 사용하기 어려운 상황이기에 현실과 타협하여 barplot으로 갑니다.
#barplot을 위한 groupby에는 이분의 코드를 참고했습니다. https://tommyhouse.oopy.io/cs_eda

groupby_genre = df.groupby('Genre', as_index = False).sum()
groupby_genre

NA_Prefer = groupby_genre.loc[1:,['Genre','NA_Sales']].sort_values(by = 'NA_Sales',ascending = False).reset_index(drop = True)
EU_Prefer = groupby_genre.loc[1:,['Genre','EU_Sales']].sort_values(by = 'EU_Sales',ascending = False).reset_index(drop = True)
JP_Prefer = groupby_genre.loc[1:,['Genre','JP_Sales']].sort_values(by = 'JP_Sales',ascending = False).reset_index(drop = True)
Other_Prefer = groupby_genre.loc[1:,['Genre','Other_Sales']].sort_values(by = 'Other_Sales',ascending = False).reset_index(drop = True)

fig, ax = plt.subplots(ncols=2,nrows=2, figsize = (30,15))
ax[0, 0].set_title("NA_Sales")
ax[0, 1].set_title("EU_Sales")
ax[1, 0].set_title("JP_Sales")
ax[1, 1].set_title("Other_Sales")
sns.barplot(data = NA_Prefer, x = 'Genre', y = 'NA_Sales', ax=ax[0, 0])
sns.barplot(data = EU_Prefer, x = 'Genre', y = 'EU_Sales', ax=ax[0, 1])
sns.barplot(data = JP_Prefer, x = 'Genre', y = 'JP_Sales', ax=ax[1, 0])
sns.barplot(data = Other_Prefer, x = 'Genre', y = 'Other_Sales', ax=ax[1, 1])

In [ ]:
groupby_genre

요약. 
* 북미는 액션이 1위, 그 뒤로 스포츠, 슈터 등이 강세이고 게임 시장이 크고 치열하네요. (잘팔리는 게임이 타 국가보다 많음)

* 유럽은 액션,스포츠,슈터, 레이싱 (북미랑 비슷) 그러나 조금 더 작고 널널한 게임시장 (잘팔리는 게임이 북미보다는 많지 않음)

* 일본은 롤플레잉이 압도적으로 강세를 보이고 있으나 피크가 높지는 않고, 다른 분야는 게임 시장이 비교적 작네요.
* 기타지역도 액션,스포츠,슈터 순이나 인기 차이가 서로 크치 않고 시장이 아주 작네요.

글로벌하고 안정적인 인기 요소를 액션,스포츠,슈터의 어떤 면이 유발했을지를 고려하여 혼합하거나, 채택하면 수익성이 더 좋겠네요. (일본은 rpg를 우선적으로 고려하면 좋겠네요.)

In [ ]:
#과제 2번.연도별 인기 게임 분류 및 시각화를 해봅시다.
df['Year'] = df['Year'].astype(int)
bins = list(range(1980, 2021, 10))
bins_label = [str(x)+"-"+str(x+10) for x in bins]
bins_label
df['YearsBy10'] = pd.cut(df['Year'].astype(int), bins, right=False, labels=bins_label[:-1])

YearInfo = df.loc[:,['Year','Genre', 'total_sum']].reset_index(drop = True)
YearInfo['YearsBy10'] = pd.cut(YearInfo['Year'].astype(int), bins, right=False, labels=bins_label[:-1])
groupby_year = YearInfo.groupby(["YearsBy10", "Genre"], as_index = False).sum()
del groupby_year['Year']
groupby_year

#잘 됐네요.

In [ ]:
#각각의 연도대별로 그린 plot
fig, ax = plt.subplots(ncols=2,nrows=2, figsize = (30,15))
graphCount = 0;

for year in bins_label[:-1]:
  i = graphCount // 2;
  j = graphCount % 2;
  ax[i, j].set_title(year)
  yearData = groupby_year[groupby_year['YearsBy10'] == year]
  yearData = yearData.loc[:,['Genre','total_sum']].sort_values(by = 'total_sum',ascending = False).reset_index(drop = True)
  idx_removed = yearData[yearData['Genre'] == 0].index
  yearData = yearData.drop(idx_removed)
  sns.barplot(data = yearData, x = 'Genre', y = 'total_sum', ax = ax[i,j])
  graphCount += 1;

이번엔 연대별 인기 장르를 분석하여 장르들의 현재와 포텐셜을 알아봅시다.

- 1980년부터 10년, 시장은 플랫폼 게임이 휩쓸었죠. 
그 중심에는 닌텐도 사의 슈퍼마리오가 있었습니다.

- 1990년부터 10년, 플랫폼은 여전하고, rpg가 성장했습니다. 
그 중심에는 포켓몬이 있었고요. 시장이 점점 커지네요?

- 2000년에 들어섰습니다. 게임 시장이 굉장히 커졌고, 상승하던 액션 장르와 닌텐도사의 WII가 견인한 스포츠 장르가 정점을 찍었죠.

- 2010년부터 지금입니다. 시장이 다소 죽었고, 액션 게임 인기는 여전하네요. 슈터 장르가 새로운 강자인데, 콜오브듀티가 견인했습니다.


In [ ]:
#과제 3번.출고량이 높은 게임들의 장르 및 연도 분류 및 시각화를 해봅시다.
#그래프 구성 참고: https://github.com/aytekin827/AIB_sec1_project/blob/main/AI_03_%EC%A0%95%EB%AF%BC%EC%9E%AC_section1.ipynb

index_of_zero = df[df['Genre'] == '0'].index
df= df.drop(index_of_zero)
# 출고량 top100의 장르, 플랫폼, 연대,출판회사
top70 = df.nlargest(70,columns='total_sum')

fig, ax = plt.subplots(ncols=2,nrows=2, figsize = (40,15))
ax[0, 0].set_title('top70 Publisher',fontsize=15)
ax[0, 1].set_title('top70 Genre',fontsize=15)
ax[1, 0].set_title('top70 Platform',fontsize=15)
ax[1, 1].set_title('all-time top70',fontsize=15)
sns.barplot(data = df, x = top70['Publisher'].value_counts().index, y = top70['Publisher'].value_counts(), ax=ax[0, 0])
sns.barplot(data = df, x = top70['Genre'].value_counts().index, y = top70['Genre'].value_counts(), ax=ax[0, 1])
sns.barplot(data = df, x = top70['Platform'].value_counts().index, y = top70['Platform'].value_counts(), ax=ax[1, 0])
sns.barplot(data = df, x = top70['Year'].value_counts().index, y = top70['Year'].value_counts(), ax=ax[1, 1])
plt.show()

출고량 상위 70개 게임 분석으로 시장을 다각적으로 알아보고, 진짜 돈 되는 플랫폼/배급사/장르를 한눈에 알아볼까요?

- 좌상단은 배급사별 누적 출고량 비교 그래프입니다. 닌텐도사가 압도적이네요. 

- 아래는 TO70게임의 게임 플랫폼 분류 그래프인데, 언뜻 X360이 매력적으로 보이나 TOP5중 3개 플랫폼이 닌텐도의 것입니다.
호환성도 좋은 닌텐도사의 플랫폼, 매력적이네요? 

- 우상단의 실적 TOP5 장르를 보시면, rpg 순위는 4위로 점유율 12.8에 그칩니다. 현재의 우리 rpg 미연시는 큰 돈이 안된다는거죠. 
장르별 매출 1위, 슈터장르도 눈여겨 볼 필요가 있겠네요.

-아래는 연도별 출고량 변화 그래프인데, 
현재 시장이 다소 죽었네요.


In [ ]:
#모든 연도대가 다 있는 plot
plt.figure(figsize = (15,15))
sns.barplot(data = groupby_year, x = "YearsBy10", y = "total_sum", hue="Genre")

전반적인 추세를 보기 위해, 연대별 장르별 출고량 그래프를 봅시다.

- 시장이 확 떴다가 가라앉았는데,액션과 스포츠, 슈터는 나름 늘 굳건하네요. 역시 돈을 부르는 장르들. 기억해둡시다.

- '실내 중심 활동의 시대'인데, 현재 스포츠의 인기가 시들해졌지만 사실 그 어느때보다 운동이 필요한 시기입니다. 자극 요소가 떨어져 시들해진 스포츠. 자극을 더해보면 어떨까요?

- 플랫폼/배급사와 무관하게 성공하는 액션 장르는 특히 눈여겨봐야 합니다.

- 미연시에 흔히 더하는 퍼즐/전략 요소는 흥행성이 떨어지니 신중하게 선택해야겠네요.

In [ ]:
#top5 장르만 뽑아 ppt로 깔끔하게 시각화하기 위한 분석
top70['Genre'].value_counts()

In [ ]:
top70['Platform'].value_counts()

결론:

이러한 분석 아래 신작 게임을 하나 제안합니다.

- 여기부터는 ppt를 같이 보셔야 이해가 됩니다. 링크: https://drive.google.com/drive/u/0/folders/1wq2E-aPrSJXcdnEgfqlxskIAp5fGdII4


- 2D화면에서가 아닌 실제로 미소녀(미소년)을 만나는 느낌의 닌텐도 Wii 기반 리얼 타임 액션 슈팅 rpg!

- 주인공은 위험천만한 현대 도시의 악당들로부터 평범해보이는, 그러나 반전의 정체를 지닌 그,그녀들을 지키는 숨겨진 히어로가 됩니다. 이것은 일상성과 새로움, 자극을 게임에 부여합니다.

- 메인 플랫폼은 닌텐도 Wii 입니다. 닌텐도 Wii를 메인으로 선정한 이유는 신체 연동 디바이스를 이용하여, 전투시 디바이스를 통한 짜릿한 사격, 무빙, 오감 플레이로 액션과 슈터의 흥행 요소를 몰입감 있게 녹여내기 위해서입니다. 또한 닌텐도 사의 충성 고객들을 만나 DS 및 닌텐도 소속 타 플랫폼들과의 확장 기반을 다질 수 있습니다. 닌텐도 ds라인의 기기들에 콘솔을 연결할 시 Wii과 유사한 환경으로 플레이할 수 있어, 적은 개발 시간과 비용으로 여러 개의 플랫폼을 공략할 수 있습니다.   

- 진출 시장은 효율적인 자금 운용을 위해 파이가 큰 북미-유럽 시장으로 제한합니다.


- 잠재 고객 확대를 위한 미연시의 인식 개선에 대해서도 고려해야합니다. 미연시 인식 설문 조사 결과에 따르면(ppt첨부) 부정 응답이 42.8%, 긍정과 중립이 나머지 반반인데, 부정 응답 요소를 줄이고, 중립을 긍정으로 전환시킨다면 더 좋은 매출이 있을겁니다.

- 이를 위해 미연시의 '과한 에로'는 줄이고, '교감' 요소는 부각합니다. 현실의 움직임을 반영하는 액션게임은 운동이죠? 클릭만 하면 성사되는 사랑이 아닌, 직접 몸으로 뛰고 운동하여 악당을 물리치고 사랑을 쟁취하는 흐름으로 비현실성을 낮추고 건강과 성취감을 함께 제공합니다.

- 일반적인 복장과 일상적인 대화 내용으로 미소녀 연애 게임에 대한 거부감을 없애고, 호감도와 시간에 따라 변화되는 복장과 리액션, 스토리로 새로움을 줍니다.

-기존 작품과 신작의 차별점에 대한 설명입니다.
첫째로. 기존 작품들은 연애에만 치중해 스토리 고갈이 빠르고 진부했고, 유저 이탈이 빨라 콘텐츠 수명이 짧았죠.

- 신작은 악당과의 진짜 액션 및 가상 리얼 타임 라이프 환경 등 콘텐츠가 풍성하고, 러브 엔딩 후에도 신혼 생활이 기다립니다.


- 둘째로, 기존작들은 인간관계가 닫혀 있어, 동시에 여자를 넷 만나도 전혀 문제가 없고, 그녀는 기계같았죠.

- 신작은 외간 여자와 달콤한 사랑을 대놓고 속삭이면 그녀의 분노가 찾아옵니다. 나의 성장과 정성, 시간에 따라 그녀의 눈빛과 스토리, 복장, 많은 것들이 달라지죠.

- 셋째로, 기존작들은 미연시가 부끄러워 남몰래 골방에서 플레이해야 했고, 안좋은 자세로 거북목 환자가 급증했죠.

- 신작은 운동 게임인 척, 엄마가 문을 열고 들어오면 빠르게 나만의 체련장으로 진입이 가능해, 자유롭고 건강한 플레이가 가능합니다. 운동 시 매력 능력치가 올라가는 것은 덤이고, 이것이 가벼운 스포츠 콘텐츠죠.

- 수익성의 측면: 신사업이고 미연시 불모지인 만큼, 초기 2년은 비용이 크게 발생되겠지만 이득은 더 커보입니다.

- 세 국가의 총 출고량을 더해 비율을 표현한 그래프를 보시면(ppt 첨부),
일단 유럽과 북미는 일본과 약 7.5배 규모 차이의 큰 시장이죠. 돈이 모이는 곳이기에 연구하여 공략할 가치가 있습니다.

- 게다가 닌텐도 사는 충성 고객이 많습니다. ppt의 세부 포인트 분석 2에서도 봤듯이, 닌텐도사의 인기작들 중 상당수가 확장/스핀오프 팩 등으로 10년 이상 매출을 올렸죠.

- 즉 북미 / 유럽시장에 닌텐도사의 플랫폼으로 진출 성공한다면, 이는 '제대로 만들고 오래 버는 길'이기에, 투자의 가치가 있다는 판단입니다.